# Media markdown generator for academicpages

Takes a TSV of media with metadata and converts them for use with [adrianarotaru.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `media.py`. Run either from the `markdown_generator` folder after replacing `media.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases, rather than Stuart's non-standard TSV format and citation style.

In [1]:
import pandas as pd
import os

## Data format

The TSV needs to have the following columns: title, type, url_slug, venue, date, location, article_url, description, with a header at the top. Many of these fields can be blank, but the columns must be in the TSV.

- Fields that cannot be blank: `title`, `url_slug`, `date`. All else can be blank. `type` defaults to "Article" 
- `date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. 
    - The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/media/YYYY-MM-DD-[url_slug]`
    - The combination of `url_slug` and `date` must be unique, as it will be the basis for your filenames

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [2]:
!cat media.tsv

title	type	url_slug	venue	date	location	article_url	description	image
EU parliament visit, at the invitation of Siegfried Muresanu, Current Vice-President of the European People's Party	Article	EU-visit	EU Parliament Visit	2016-04-19	Brussels, Belgium	https://diez.md/2016/04/19/foto-tinerele-lidere-de-la-leadershe-young-womens-academy-au-vizitat-parlamentul-european/	According a report released by the Moldovan National Bureau of Statistics, Women and girls occupy only 31% of jobs in the ICT sector in Moldova and only 19% of digital professions. As an advocate for women representation in science and equal gender rights in Moldova, I have been invited by members of the EU Parliament to participate to the dialogue about policy making addressing representation of women in science and politics.	../images/visit.png
Nomination for the "Best Bachelor's Student Abroad" by the Gala of Students Originating from Moldova, VII-th Edition	Article	best-student	Gala of Students Originating from Moldova

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [3]:
media = pd.read_csv("media.tsv", sep="\t", header=0)
media

,title,type,url_slug,venue,date,location,article_url,description,image
0,"EU parliament visit, at the invitation of Sieg...",Article,EU-visit,EU Parliament Visit,2016-04-19,"Brussels, Belgium",https://diez.md/2016/04/19/foto-tinerele-lider...,According a report released by the Moldovan Na...,../images/visit.png
1,"Nomination for the ""Best Bachelor's Student Ab...",Article,best-student,Gala of Students Originating from Moldova,2019-09-15,"Chisinau, Moldova",https://unica.md/monden/lista-celor-mai-buni-s...,The Gala of Students Originating from Moldova ...,../images/gala.png


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [4]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    if type(text) is str:
        return "".join(html_escape_table.get(c,c) for c in text)
    else:
        return "False"

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [5]:
loc_dict = {}

for row, item in media.iterrows():
    
    md_filename = str(item.date) + "-" + item.url_slug + ".md"
    html_filename = str(item.date) + "-" + item.url_slug 
    year = item.date[:4]
    
    md = "---\ntitle: \""   + item.title + '"\n'
    md += "collection: media" + "\n"
    
    if len(str(item.type)) > 3:
        md += 'type: "' + item.type + '"\n'
    else:
        md += 'type: "Article"\n'
    
    md += "permalink: /media/" + html_filename + "\n"
    
    if len(str(item.venue)) > 3:
        md += 'venue: "' + item.venue + '"\n'
        
    if len(str(item.location)) > 3:
        md += "date: " + str(item.date) + "\n"
    
    if len(str(item.location)) > 3:
        md += 'location: "' + str(item.location) + '"\n'
           
    md += "---\n"
    
    
    if len(str(item.article_url)) > 3:
        md += "\n[More information here](" + item.article_url + ")\n" 
        
    
    if len(str(item.description)) > 3:
        md += "\n" + html_escape(item.description) + "\n"
        
    if len(str(item.image)) > 3: 
        md +=f"\n![Image]({item.image})\n"     
    md_filename = os.path.basename(md_filename)
    #print(md)
    
    with open("../_media/" + md_filename, 'w') as f:
        f.write(md)

These files are in the media directory, one directory below where we're working from.

In [6]:
!ls ../_media

2016-04-19-EU-visit.md     2019-09-15-best-student.md


In [7]:
!cat ../_media/2013-03-01-tutorial-1.md

cat: ../_media/2013-03-01-tutorial-1.md: No such file or directory
